## Functions for TAA Component Preparation

Complementary notebook to executable on the same topic.
Functions are presented in an order closely following the exe notebook steps:

#### Libraries

In [1]:
import os
import re
import numpy as np                                      # basic numeric calculation
import pandas as pd                                     # split-apply-combine operations on dataframe
 
import fiona
import geopandas as gpd

import rasterio
import rasterstats
 
import matplotlib as mpl
import matplotlib.pyplot as plt                         # plotting tool
from matplotlib.patches import RegularPolygon           # drawing hexagons
 
import shapely                                          # to attribute geometric properties for shapes
from shapely.geometry import Polygon, Point

#### 1. General functions


In [2]:
def get_agg_cols(data, from_position):
  return list(data.columns)[from_position:]


In [3]:
def snake2camel(name):
    return re.sub(r'(?:^|_)([a-z])', lambda x: x.group(1).upper(), name)

In [4]:
def camel2spaced(name):
    return re.sub('([a-z])([A-Z])', '\\g<1> \\g<2>', name)

In [ ]:
def read_geojson(path: str, file_name: str):
  """
    A function to read a geojson file 
    
    Arguments: 
    path - address to directory to read from
    file_name - name of file to read
 
    Returns: GeoDataFrame as read from geojson file
  """
    
  file = open(os.path.join(path, file_name))
  df = gpd.read_file(file)
  df = df.set_crs(epsg=4326)
  
  return df

In [5]:
def read_geojson_(path: str, file_name: str):
  """
    A function to read a geojson file 
    
    Arguments: 
    path - address to directory to read from
    file_name - name of file to read
 
    Returns: GeoDataFrame as read from geojson file
  """
    
  file = open(os.path.join(path, file_name),encoding="utf-8")
  df = gpd.read_file(file)
  df = df.set_crs(epsg=4326)
  
  return df

In [6]:
def _to_2d(x, y, z):
    return tuple(filter(None, [x, y]))

In [7]:
def construct_wmean_func(carto_data_name, cred_container, w_col):
  """
    A function to create a weighted mean function object when given input specs.
    
    Arguments: 
    carto_data_name {string} -- A string containing the name of a carto dataset (for live read)
    cred_container {string} -- A string containing the file path and name of a json on carto credentials (for live read)
    w_col {string} -- A string containing the name of a column to use for weighing
 
    Returns: function -- 
  """  
  
  carto_data = ingest_carto_data(carto_data_name, cred_container)
  wmean = lambda x: np.average(x, weights=carto_data.loc[x.index, w_col])
  wmean.__name__ = "wmean"
  
  return wmean


In [8]:
def construct_wmean_func_from_set(data, w_col):
  """
    A function to create a weighted mean function object when given input specs.
    
    Arguments: 
    data {gpd.DataFrame} -- A dataframe to index on for custom wmean function
    w_col {string} -- A string containing the name of a column to use for weighing
 
    Returns: function -- 
  """  
  
  wmean = lambda x: np.average(x, weights=data.loc[x.index, w_col])
  wmean.__name__ = "wmean"
  
  return wmean


In [9]:
def derive_outlets_by_chan(cmd, geo_data, id_var="CUSTOMER_ID"):
  """
    A function to engineer features for the number of outlets - by type for geographical grouping (hexagons, or trade areas).
    
    Arguments: 
    cmd {SparkDataFrame} -- WP population data - aggregated to hexagon level
    geo_data {GeoDataFrame} -- OSM poi data - aggregated to hexagon level
    id_var {str} -- Name of customer id variable
 
    Returns: Pandas DataFrame with columns for the number of outlet types within each hexagon/trade area.
  """     
  
  # 1) Define channel subcategories
  outlets_by_chan = cmd. \
    select(*[id_var,"CHANNEL_CUSTOM", "LATITUDE", "LONGITUDE"]). \
    toPandas()
 
 
  outlets_by_chan['CHANNEL_COUNTS'] = outlets_by_chan['CHANNEL_CUSTOM']

  outlets = outlets_by_chan["CHANNEL_COUNTS"].unique().tolist()
  
  outlets_by_chan = outlets_by_chan.set_index([id_var, "LATITUDE", "LONGITUDE"])
  outlets_by_chan = pd.get_dummies(outlets_by_chan['CHANNEL_COUNTS']).reset_index()
  outlets_by_chan = gpd.GeoDataFrame(outlets_by_chan, geometry=gpd.points_from_xy(outlets_by_chan["LONGITUDE"], outlets_by_chan["LATITUDE"]), crs="epsg:4326")
  
  # 2) Aggregate to hexagon level 
#   outlets_by_chan_ready_set = pd.merge(outlets_by_chan, geo_data, on=id_var, how="left")
  outlets_by_chan_ready_set = gpd.overlay(outlets_by_chan, geo_data, how="intersection")
  
  outlets_by_chan_ready_set = outlets_by_chan_ready_set. \
    groupby("hex_id"). \
    agg(dict(zip(outlets, ["sum"]*len(outlets)))). \
    reset_index()
  
  return outlets_by_chan_ready_set



#### 2. Functions on preparing & overlaying a hexagonal grid over a territory

In [10]:
def haversine_custom(coord1, coord2):
  """
      A function to determine the great-circle distance between 2 points on Earth given their longitudes and latitudes
 
      Arguments: 
      coord1 - territory bounds for first point, lon & lat
      coord2 - territory bounds for second point, lon & lat
 
      Returns: Distance in meters
  """ 
  
  # Coordinates in decimal degrees (e.g. 43.60, -79.49)
  lon1, lat1 = coord1
  lon2, lat2 = coord2
 
  # Radius of Earth in meters
  R = 6371000  
 
  phi_1 = np.radians(lat1)
  phi_2 = np.radians(lat2)
  delta_phi = np.radians(lat2 - lat1)
  delta_lambda = np.radians(lon2 - lon1)
 
  a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi_1) * np.cos(phi_2) * np.sin(delta_lambda / 2.0) ** 2
  c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1 - a))
 
  # Output distance in meters
  meters = R * c  
 
  # Output distance in kilometers
  km = meters / 1000.0  
  meters = round(meters)
  km = round(km, 3)
 
  #print(f"Distance: {meters} m")
  #print(f"Distance: {km} km")
 
  return meters

In [11]:
 def form_hex_grid(territory, hex_diameter: int):
  """
    A function to form a hexagonal grid
    
    Arguments: 
    territory - GeoDataFrame for a territory
    hex_diameter - integer, diameter size to use in defining hexagon dimensions
 
    Returns: GeoDataFrame with geometry for hexagonal grid formed for a territory provided
  """ 
  
  # 1) Define general hex parameters
  xmin, ymin, xmax, ymax = [x * 1.01 for x in territory.total_bounds]
  EW = haversine_custom((xmin,ymin),(xmax,ymin))
  NS = haversine_custom((xmin,ymin),(xmin,ymax))
  
  # diamter of each hexagon in the grid
  d = hex_diameter
  
  # horizontal width of hexagon = w = d* sin(60)
  w = d*np.sin(np.pi/3)
  
  # Approximate number of hexagons per row = EW/w 
  n_cols = int(EW/w) + 1
  
  # Approximate number of hexagons per column = NS/d
  n_rows = int(NS/w) + 1
  
  # 2) Add hex params to territory
  
  # ax = territory[["geometry"]].boundary.plot(edgecolor='black', figsize=(30, 60))  #
  
  # width of hexagon
  w = (xmax-xmin)/n_cols 
  
  # diameter of hexagon
  d = w/np.sin(np.pi/3)
  
  array_of_hexes = []
  for rows in range(0,n_rows):
      hcoord = np.arange(xmin,xmax,w) + (rows%2)*w/2
      vcoord = [ymax- rows*d*0.75]*n_cols
      for x, y in zip(hcoord, vcoord):  #, colors):
          hexes = RegularPolygon((x, y), numVertices=6, radius=d/2, alpha=0.2, edgecolor='k')
          verts = hexes.get_path().vertices
          trans = hexes.get_patch_transform()
          points = trans.transform(verts)
          array_of_hexes.append(Polygon(points))
          # ax.add_patch(hexes)  #
 
  # ax.set_xlim([xmin, xmax])  #
  # ax.set_ylim([ymin, ymax])  #
  # plt.show()  #
  
  # 3) Form hex grid as gpd
  hex_grid = gpd.GeoDataFrame({'geometry': array_of_hexes}, crs="EPSG:4326")
  hex_grid = hex_grid.to_crs(epsg=4326)
  
  return hex_grid


In [12]:
def hexalize_territory(territory, hex_grid):
  """
    A function to add hexagonal grid geometry to GeoDataFrame territory 
    
    Arguments: 
    territory - GeoDataFrame for a territory
    hex_grid - GeoDataFrame, hexagonal grid geometry as prepared for specified territory
 
    Returns: GeoDataFrame of a territory overlayed with a hexagonal grid
  """ 
  
  territory_hex = gpd.overlay(hex_grid, territory)
  territory_hex = gpd.GeoDataFrame(territory_hex, geometry='geometry', crs="EPSG:4326")
  territory_hex = territory_hex.reset_index()
  territory_hex.rename(columns={'index': 'hex_id'}, inplace=True)
  
  return territory_hex


In [13]:
def derive_hex_area(data):
  """
    A function to derive the square kilometer area per hexagon 
    
    Arguments: 
    data - GeoDataFrame to calculate the area for pre-specified polygons (hexagons)
 
    Returns: GeoDataFrame expanded with a field on square kilometer are per hexagon
  """
     
  data['hexagon_area_sq_m'] = data.to_crs({'proj':'cea'})["geometry"].area
  data['hexagon_area_sq_km'] = data['hexagon_area_sq_m'] / 1000000  
  
#   data = data.drop(['hexagon_area_share', 'hexagon_area'], axis=1)
  data = data.to_crs(epsg=4326)
  
  return data


In [14]:
def prep_territory_hex(hex_diameter: int = 2500,
                       territory_scope: str = '/dbfs/mnt/AA/ba008/assets/shape_files/morocco_l2_adm_areas.shp',
                       hex_set: str = None):
  """
    A function to overlay a custom hexagonal grid over a territory
    
    Arguments:
    hex_diameter - integer, diameter size to use in defining hexagon dimensions
    territory_scope - string, name of shape file to draw areas to keep for the analysis (i.e. only marrakech_eccbc), default is set to full country
    hex_set - string, name of json file to export
    
    Returns: GeoJSON file saved to fs - of a territory overlayed with a customizable hexagonal grid
  """ 
    
    
  # 1) Load full territory data, i.e. from country shape file
  territory = gpd.read_file(territory_scope)
  territory = territory.to_crs(epsg=4326)
    
  # 2) Form hex grid for the relevant territory - post area selection
  hex_grid = form_hex_grid(territory, hex_diameter)
  
  # 3) Add hexagonal grid to territory
  territory_hex = hexalize_territory(territory, hex_grid)
  
  # 4) Add a field for the area per hexagon
  territory_hex = derive_hex_area(territory_hex)
  
  # 5) Export
  if hex_set is not None:
    territory_hex.to_file(hex_set, driver='GeoJSON')
  
  return territory_hex

#### 3. Functions on preparinging external data sources and performing hexagonal-based aggregations

###### 3.1.1. Wrapper function on population data, source: Worldpop 

In [15]:
def prep_pop_set(path_hex: str, hex_set: str, path_pop: str, pop_set: str, agg_funcs: list, export_agg_file: str):
  """
    A function to derive aggregate population stats by hexagon
    
    Arguments: 
    path_hex - string, name of fs directory to read hex data from
    hex_set - string, name of shape file containing hexagon grid (read as gpd)
    path_pop - string, name of fs directory to read population data from
    pop_set - string, name of tif file with population data (read as raster)
    agg_funcs - list of strings with aggregation operations to be performed over the population data
    export_agg_file - file path to export aggregated data to
 
    Returns: GeoDataFrame with population stats aggregated by hexagon
  """
  
  set_prefix = 'pop_'
  
  # 1) Derive aggregated stats: over the raster data
  output = gpd.GeoDataFrame.from_features(
  rasterstats.zonal_stats(
    read_geojson(path_hex, hex_set),
    os.path.join(path_pop, pop_set),
    stats=agg_funcs,
    prefix=set_prefix,
    geojson_out=True))
  
  # 2) Fix: a few NaN-s arise from the aggregation step -> replace with 0s
  for i in range(0, len(agg_funcs), 1):
    output[set_prefix+agg_funcs[i]].fillna(0, inplace=True)
  
  # 3) Set the coordinate reference system
  output.set_crs(epsg=4326)
  
  # 4) Export to blob
  output.to_file(export_agg_file, driver='GeoJSON')
  
  return output


###### 3.1.2 Wrapper function on osm poi data, source: OSM

In [16]:
def prep_osm_set(territory_hex: gpd.GeoDataFrame, path_osm: str, osm_set: str, export_agg_file: str = None):
  """
    A function to derive the point-of-interest number per hexagon (count)
    
    Arguments: 
    path_hex - string, name of fs directory to read hex data from
    hex_set - string, name of shape file containing hexagon grid (read as gdp)
    path_osm - string, name of fs directory to read osm data from
    osm_set - string, name of parquet file with osm data (read as pd)
    export_agg_file - string, file path to export aggregated data to
 
    Returns: GeoDataFrame with the number of poi-s per hexagon
  """  
  
  # 1) Read osm set
  init =  pd.read_parquet(os.path.join(path_osm, osm_set)).filter(["osm_id", "latitude", "longitude"], axis=1)
  output = gpd.GeoDataFrame(init, geometry=gpd.points_from_xy(init.longitude, init.latitude))
  output = output.set_crs(epsg=4326)
  
  # 2) Add hex grid 
  output = gpd.sjoin(output, territory_hex, how="inner", op="intersects")
  
  # 3) Derive aggregated stats: over the goepandas data
  output = output.groupby(['hex_id'])[['osm_id']].count()
  output.reset_index(level=0, inplace=True)
  output.rename(columns={'osm_id': 'osm_count'}, inplace=True)
  
  # 4) Export
  if export_agg_file is not None:
    output.to_parquet(export_agg_file)
    
  return output


###### 3.1.3. Wrapper function on customer spend data, source: Carto/Experian

In [17]:
def prep_carto_cexpend(territory_hex: gpd.GeoDataFrame, carto_data):
  """
    A function to derive carto aggregations per hexagon
    
    Arguments: 
    territory_hex {gpd.GeoDataFrame} -- The hexagonal coverage of the territory
    carto_data {pd.DataFrame} -- Carto data read in the notebook
 
    Returns: GeoDataFrame with carto hexagon-level aggregations
  """  
  
  # 1) Set helper items
  val_cols = ["wvce_01"]  ## --- may need changing -> expecting a list of column names 
  aggs = [["sum"]] * len(val_cols)                           ## --- may need changing -> expecting a list of str-lists on function names (for naming @ step7)
  
  # 2) Read input sets
  # carto_data = ingest_carto_data(carto_data_name, cred_container)
#   territory_hex = read_geojson(path_hex, hex_set)
  
  # 3) Overlay carto and hexagonal grid
  output = overlay_carto_hex(carto_data, territory_hex, carto_id_var="cartodb_id", hex_id_var="hex_id")
  
  # 4) Update columns to reflect the carto value share within each hexagon
  output = reflect_carto_share(output, val_cols)
   
  # 5) Derive aggregated stats
  output = output. \
    groupby("hex_id"). \
    agg(dict(zip(val_cols, aggs)))
  
  # 6) Address naming
  output = fix_agg_data_names(output, prefix="carto_", id_var="hex_id")
 
  return output

###### 3.1.4. Wrapper function on socio-demographic data, source: Carto/Experian

In [37]:
def prep_carto_sociodemo(territory_hex: gpd.GeoDataFrame, carto_data, hex_geom_name = "territory_hex_geom", carto_geom_name = "carto_set_geom"):
  """
    A function to derive carto aggregations per hexagon
    
    Arguments: 
    territory_hex {gpd.GeoDataFrame} -- The hexagonal coverage of the territory
    carto_data {pd.DataFrame} -- Carto data read in the notebook
 
    Returns: GeoDataFrame with carto hexagon-level aggregations
  """  
  
  # 1) Set helper items
  val_cols =['pop','hh','male','female','age_t0014', 'age_m0014', 'age_f0014', 'age_t1529', 'age_m1529','age_f1529','age_t3044','age_m3044','age_f3044','age_t4559','age_m4559','age_f4559','age_t60pl','age_m60pl','age_f60pl','di_mio','di_pc','di_ci']
  # 2) Read input sets
  # carto_data = ingest_carto_data(carto_data_name, cred_container)
#   territory_hex = read_geojson(path_hex, hex_set)
  
  # 3) Overlay carto and hexagonal grid
  output = overlay_carto_hex(carto_data, territory_hex, carto_id_var="cartodb_id", hex_id_var="hex_id")
  
  # Add aggregation functions - data needed for construct_wmean_func_from_set
  aggs = [["sum"]]*20 + [["mean", construct_wmean_func_from_set(output, "pop")]]*2   ## --- may need changing -> expecting a list of str-lists on function names (for naming @ step7)
  
  # 4) Update columns to reflect the carto value share within each hexagon
  output = reflect_carto_share(output, val_cols, hex_geom_name, carto_geom_name)
   
  # 5) Derive aggregated stats
  output = output. \
    groupby("hex_id"). \
    agg(dict(zip(val_cols, aggs)))
  
  # 6) Address naming
  output = fix_agg_data_names(output, prefix="carto_", id_var="hex_id")
 
  return output


###### 3.1.5. Wrapper function on activity/mobility data, source: Unacast

In [19]:
def prep_carto_unacast(territory_hex: gpd.GeoDataFrame, carto_data):
  """
    A function to derive carto aggregations per hexagon
    
    Arguments: 
    territory_hex {gpd.GeoDataFrame} -- The hexagonal coverage of the territory
    carto_data {pd.DataFrame} -- Carto data read in the notebook
 
    Returns: GeoDataFrame with carto hexagon-level aggregations
  """    
    
  # 1) Read input sets
  # 1.1) Territory hex set 
#   territory_hex = read_geojson(path_hex, hex_set)
  # 1.2) Carto set
  # carto_data = ingest_carto_data(carto_data_name, cred_container)
  carto_data = init_prep_carto_unacast(carto_data, id_var="seen_in")
  
  # 2) Set helper items
  # val_cols = get_agg_cols(data=carto_data, from_position=2)  ## --- may need changing -> expecting a list of column names (vestige)
  vals_sum = list(carto_data[carto_data.filter(regex='staying').columns]) 
  vals_mean = list(carto_data[carto_data.filter(regex='proportion_|return_').columns])
  val_cols = vals_sum + vals_mean
  aggs = [["sum"]]*len(vals_sum) + [["mean"]]*len(vals_mean)   ## --- may need changing -> expecting a list of str-lists on function names (for naming @ step7)
  
  # 3) Overlay carto and hexagonal grid
  output = overlay_carto_hex(carto_data, territory_hex, carto_id_var="seen_in", hex_id_var="hex_id")
  
  # 4) Update columns to reflect the carto value share within each hexagon
  # AR: There's something wrong with this function
  output = reflect_carto_share(output, val_cols)
   
  # 5) Derive aggregated stats
  output = output. \
    groupby("hex_id"). \
    agg(dict(zip(val_cols, aggs)))
  
  # 7) Address naming
  output = fix_agg_data_names(output, prefix="carto_", id_var="hex_id")
  
  return output


###### 3.1.6. Wrapper function on poi data, source: Pitney Bowes

In [20]:
def prep_carto_poi(territory_hex: gpd.GeoDataFrame, carto_data, carto_val_cols):
  """
    A function to derive the carto point-of-interest number per hexagon (count), item/s to count may be changed if needed.
    
    Arguments: 
    territory_hex {gpd.GeoDataFrame} -- The hexagonal coverage of the territory
    carto_data {pd.DataFrame} -- Carto data
    carto_val_cols {str list} -- Names of carto columns to aggregate 
 
    Returns: GeoDataFrame with the number of poi-s per hexagon
  """  
  
  # 0) Helper items
  aggs = [["sum"]]*len(carto_val_cols)  # summing dummies -->> resulting in counts
  
  # 1) Aggregate
  output = \
    gpd.sjoin(custom_group_carto_poi(carto_data), territory_hex, how="inner", op="intersects"). \
    groupby("hex_id"). \
    agg(dict(zip(carto_val_cols, aggs)))
 
  # 2) Adjust names
  output = fix_agg_data_names(output, prefix="carto_", id_var="hex_id")
  output.columns = output.columns.str.replace("sum", "count")
      
  return output


###### 3.2.1. Additional general function

In [21]:
def overlay_carto_hex(carto_data, territory_hex, carto_id_var, hex_id_var):
  """
    A function to overlay carto data and hexagonal grid
    
    Arguments: 
    carto_data {pd.DataFrame} -- Carto data
    territory_hex {pd.DataFrame} -- Hexagonal grid data for the territory
    carto_id_var {string} -- Name of carto_data id column
    hex_id_var {string} -- Name of territory_hex id column
 
    Returns: GeoDataFrame with carto hexagon-level aggregations
  """  
  
  # 1) Overlay hex grid onto carto data -->> NB! the default geometry becomes that of the intersection items
  output = gpd.overlay(carto_data, territory_hex, how="intersection")
  
  # 2) Add extra geometries - from full carto set and from full hex set (gives context for carto value weighing)
  output = pd.merge(output, carto_data[[carto_id_var, "geometry"]].rename(columns={"geometry": "carto_set_geom"}), on = [carto_id_var], how="left")
  output = pd.merge(output, territory_hex[[hex_id_var, "geometry"]].rename(columns={"geometry": "territory_hex_geom"}), on = [hex_id_var], how="left")
  
  return output


###### 3.2.2. Additional general function

In [22]:
def reflect_carto_share(data, val_cols, hex_geom_name = "territory_hex_geom", carto_geom_name = "carto_set_geom"):
  """
    A function to update carto value columns with their approproiate carto share - falling within each hexagon, prior to follow-up aggregation
 
    Arguments: 
    data {gpd.DataFrame} -- Data for which to update the carto value columns 
    val_cols {str list} -- Name of shape file containing hexagon grid (read as gpd)
 
    Returns: GeoDataFrame with carto hexagon-level aggregations
  """  
    
#   hex_geom_name = "territory_hex_geom"
#   carto_geom_name = "carto_set_geom"
# "geometry" is the geom of the intersection from the overlay intersections
  
  form_hex_share = lambda x: [y.area/z.area for y, z in zip(x["geometry"], x[carto_geom_name])]
  
  for each in val_cols:
    form_val_as_share = lambda x: [y*z for y, z in zip(x[each], x["hex_share"])]
    new_col_spex = dict(zip(["hex_share"] + [each], [form_hex_share] + [form_val_as_share]))
    
    data = data.assign(**(new_col_spex))
    
  return data


##### 3.2.3. Additional general function

In [23]:
def fix_agg_data_names(data, prefix="carto_", id_var="hex_id"):
  """
    A function to adjust column names of aggregated data - 
    concatenates index grouping to column names and adds an indicator prefix (for easier filtering @ later stages)
    
    Arguments: 
    data {gpd.DataFrame} -- Post-pivot data to process
    prefix {string} -- Suffix to add to column names, short indicator of pivot column setting (i.e. "dow" for "day_of_week")
    id_var {string} -- Name of DataFrame id var
 
    Returns: gpd.DataFrame with adjusted column names.
  """    
    
  data.reset_index(level=0, inplace=True)
  data.columns = ["_".join(x) for x in data.columns.ravel() if x != id_var]  # [id_var] + 
  data = data.add_prefix(prefix)
  data = data.rename(columns={prefix+id_var+"_": id_var})
  data.head()
  
  return data


###### 3.2.4. Additional general function


In [24]:
def fix_pivot_data_names(data, suffix, id_var, id_trigger=False):
  """
    A function to adjust column names of pivoted data - 
    concatenates index grouping to column names and adds an indicator suffix (for easier filtering @ later stages)
    
    Arguments: 
    data {gpd.DataFrame} -- Post-pivot data to process
    suffix {string} -- Suffix to add to column names, short indicator of pivot column setting (i.e. "dow" for "day_of_week")
    id_var {string} -- Name of DataFrame id var
    id_trigger {boolean} -- Flag for id_var treatment in renaming, False drops id_var from renaming functionality (defaults to False)
 
    Returns: performs renaming in place, does not return an object
  """    
  
  if id_trigger:
    data.columns = ["_".join(x) + suffix for x in data.columns.ravel()]
  else:
    data.columns = ["_".join(x) + suffix for x in data.columns.ravel() if x != id_var]
    
  data.reset_index(level=0, inplace=True)


###### 3.2.5. Additional set-specific function: carto_poi (callable in wrapper 3.1.6 "prep_carto_poi")


In [25]:
def custom_group_carto_poi(carto_data, id_var="cartodb_id"):
  """
    A function to create a custom grouping based on pb poi trade divisions - to leverage in the estimation of poi count by category. 
    
    Arguments: 
    carto_data {pd.GeoDataFrame} -- Carto poi data to process
    id_var {string} -- Name of id var (key id) to use in indexing and df joining
 
    Returns: GeoDataFrame on carto PB POI - with dummies for the different custom poi groups defined.
  """   
  
  # 1) Create subset with trade divisions only
  data = carto_data.copy()[[id_var, "trade_division"]]
 
  # 2) Define new column parameters - conditions and names, 
  conditions = [
      (data["trade_division"].isin(["DIVISION C. - CONSTRUCTION"])),
      (data["trade_division"].isin(["DIVISION D. - MANUFACTURING"])),
      (data["trade_division"].isin(["DIVISION B. -MINING"])),

      (data["trade_division"].isin(["DIVISION A. - AGRICULTURE, FORESTRY, AND FISHING"])),
      (data["trade_division"].isin(["DIVISION K. - NONCLASSIFIABLE ESTABLISHMENTS"])),
      (data["trade_division"].isin(["DIVISION N. - LABEL FEATURES"])),
      (data["trade_division"].isin(["DIVISION E. - TRANSPORTATION AND PUBLIC UTILITIES"])),
      (data["trade_division"].isin(["DIVISION I. - SERVICES", "DIVISION F. - WHOLESALE TRADE", "DIVISION G. - RETAIL TRADE", "DIVISION M. - SPORTS",
                                    "DIVISION L. - TOURISM", "DIVISION H. - FINANCE, INSURANCE, AND REAL ESTATE", "DIVISION J. - PUBLIC ADMINISTRATION"]))
       ]
 
  values = ["poi_" + x for x in ["construction", "manufacturing","mining", "life_sciences", "nonclassifiable", "terrain_features", "utility_transportation", "trade_services"] ]
 
  # 3) Generate new columns
  data = data. \
    assign(poi_sic_grouping=np.select(conditions, values)). \
    set_index(id_var)    
 
  business = pd.get_dummies(data["poi_sic_grouping"]). \
    reset_index()
  
  divisions = pd.get_dummies(data["trade_division"])
  divisions.columns = [re.sub(r'^.*?-', "", x.lower()) for x in divisions.columns]
  divisions.columns = [re.sub(",", "", x) for x in divisions.columns]
  divisions.columns = ["trade_div" + re.sub(" ", "_", x) for x in divisions.columns]
  divisions = divisions.reset_index()
  
  # 4) Add to original carto data
  carto_data = carto_data.merge(business, on=id_var, how="left").merge(divisions, on=id_var, how="left")
  
  return carto_data


###### 3.2.6. Additional set-specific function: carto_unacast (callable in wrapper 3.1.5 "prep_carto_unacast")


In [26]:
def init_prep_carto_unacast(carto_data, id_var="seen_in"):
  """
    A function to perform initial prep over the raw carto_unacast_data - spread to wide format.
    
    Arguments: 
    carto_unacast {gpd.DataFrame} -- Spread Carto Unacast data to be processed
    id_var {string} -- Name of dataset key id var/quadid (indicating unique rows, defaults to "seen_in")
 
    Returns: GeoDataFrame with wider data.
  """  
    
  # 1) Spread carto set categorical columns 
  carto_unacast_wip = spread_carto_unacast(carto_data)
  
  # 2) Transform original proportions to counts
  carto_unacast_wip = reflect_unacast_proportions(carto_unacast_wip)
  
  # 3) Aggregate carto set to geom level
  output = agg_carto_unacast_to_geomid(carto_unacast_wip, id_var=id_var)
  
  return output


**3.2.7. Additional set-specific function: carto_unacast (callable in addon 3.2.6 "init_prep_carto_unacast")**

In [27]:
def spread_carto_unacast(carto_unacast):
  """
    A function to spread categorical carto_unacast columns
    
    Arguments: 
    carto_unacast {gpd.DataFrame} -- Carto Unacast data to be processed
 
    Returns: GeoDataFrame with wider data.
  """  
  
  # Unfold return_rate column
  try:
    carto_unacast_wip = carto_unacast. \
      assign(
        rr_unfolded = lambda x: [parse_return_rate(string=y) for y in x["return_rate"]],
        less_than_5_flag = carto_unacast["less_than_5_flag"].astype(int)
      )
  except:
    return carto_unacast
 
  # Spread into columns
  carto_unacast_wip = pd.concat([carto_unacast_wip.drop(["rr_unfolded", "return_rate"], axis=1), carto_unacast_wip["rr_unfolded"].apply(pd.Series)], axis=1)
  carto_unacast_wip.columns = ["return_rate_" + col if col in ["rare", "weekly", "multi_weekly", "monthly"] else col for col in carto_unacast_wip.columns]
 
   # Adjust type 
  fix_cols = [col for col in carto_unacast_wip if col.startswith("return_rate_")]
  carto_unacast_wip[fix_cols] = carto_unacast_wip[fix_cols].apply(pd.to_numeric, errors='coerce', axis=1)
  
  return carto_unacast_wip



**3.2.8. Additional set-specific function: carto_unacast (callable in addon 3.2.7 "spread_carto_unacast")**

In [28]:
def parse_return_rate(string):
  """
    A function to parse return_rate column of the carto unacast data (originally a string).
    
    Arguments: 
    string {string} -- String to parse
 
    Returns: String - parsed as a dictionary of key-value pairs to be split into DataFrame columns as the next prep step.
  """  
  
  output_string = string.split("), ")
  output_string = [re.sub(r'[]()[{}]', '', x) for x in output_string]
  output_string = [re.sub(r' ', '', x) for x in output_string]
  output_string = [re.sub(r'frequency:', '', x) for x in output_string]
  output_string = [re.sub(r'ratio:', '', x) for x in output_string]
  output_string = [re.sub(r'-', '_', x) for x in output_string]
  output_string = [re.sub(r',', '=', x) for x in output_string]
  
  sep = ";"
  output_string = sep.join(output_string)
  
  output_string = dict(item.split("=") for item in output_string.split(";"))
    
  return output_string

**3.2.9. Additional set-specific function: carto_unacast (callable in addon 3.2.6 "init_prep_carto_unacast")**

In [3]:
def reflect_unacast_proportions(carto_unacast_wip):
  """
    A function to reflect proportions as counts in unacast data (relative to "staying" count) - prior to aggregation.
    
    Arguments: 
    carto_unacast_wip {gpd.DataFrame} -- Spread Carto Unacast data to be processed
 
    Returns: GeoDataFrame with original proportion columns transformed as counts.
  """    
  
  fix_cols =  [col for col in carto_unacast_wip if col.startswith("proportion_")]
  
  for col in fix_cols:
    carto_unacast_wip[col] = carto_unacast_wip["staying"] * carto_unacast_wip[col]
    
  return carto_unacast_wip


**3.2.10. Additional set-specific function: carto_unacast (callable in addon 3.2.10 "init_prep_carto_unacast")**

In [30]:
def agg_carto_unacast_to_geomid(carto_unacast_wip, id_var="seen_in"):
  """
    A function to aggregate carto_unacast data to quadid level (same as geometry level).
    
    Arguments: 
    carto_unacast {gpd.DataFrame} -- Spread Carto Unacast data to be processed
    id_var {string} -- Name of dataset key id var/quadid (indicating unique rows, not a geometry)
 
    Returns: GeoDataFrame with wider data.
  """  
  
  # 1) Helper items
  geom_mapping = carto_unacast_wip[["geometry", id_var]].drop_duplicates()
  val_cols = ['staying','proportion_residents','proportion_workers','proportion_others','proportion_same_state','proportion_other_state','proportion_unknown','return_rate_rare','return_rate_multi_weekly','return_rate_weekly','return_rate_monthly']
  rr_cols = [col for col in val_cols if col.startswith("return_rate_")]
  
  # 2) Derive wide set
  
  # 2.1) aggregate return_rates to seen_in id -> independent of any breaks
  # average (if spread by category the same result is observed across different categories)
  out_rr_mean = carto_unacast_wip.groupby(id_var).agg(dict(zip(rr_cols, ["mean"]*len(rr_cols)))).reset_index()
  
  # 2.2) aggregate day_of_week
  day_of_week = \
    wide_agg_carto_unacast(
      carto_unacast_wip, val_cols, id_var, dimension_var="day_of_week", suffix_dim="_dow", 
      suffix_specs=["ALL", "Friday", "Monday", "Saturday", "Sunday", "Thursday", "Tuesday", "Wednesday"])
  
  # 2.3) aggregate time_of_day
  part_of_day = \
    wide_agg_carto_unacast(
      carto_unacast_wip, val_cols, id_var, dimension_var="part_of_day", suffix_dim="_pod", 
      suffix_specs=['ALL', 'Evening', 'Morning', 'Afternoon', 'Night'])  
  
  # 2.4) aggregate months
  month = \
    wide_agg_carto_unacast(
      carto_unacast_wip, val_cols, id_var, dimension_var="month", suffix_dim="_mo", 
      suffix_specs=['September', 'May', 'August', 'June', 'July', 'October'])  
  
  # 2.4) pivot on combination of day_of_week, time_of_day & month --> NB! results in 240 cols per value var 
  #   time_combos = carto_unacast_wip. \
  #     assign(part_of_time = lambda x: [y + "_" + z + "_" + q for y, z, q in zip(x["month"], x["day_of_week"], x["part_of_day"])]). \
  #     pivot(index=id_var, columns="part_of_time", values=val_cols)
  #   fix_pivot_data_names(data=time_combos, suffix="_combo", id_var=id_var, id_trigger=True)  
  
  # 2.5) Combine into 1
  output = geom_mapping. \
    merge(month, on=id_var, how="inner"). \
    merge(day_of_week, on=id_var, how="inner"). \
    merge(part_of_day, on=id_var, how="inner"). \
    merge(out_rr_mean, on=id_var, how="inner") #. \
    # merge(time_combos, on=id_var, how="inner")
  
  return output


**3.2.11. Additional set-specific function: carto_unacast (callable in addon 3.2.10 "agg_carto_unacast_to_geomid")**

In [31]:
def wide_agg_carto_unacast(carto_unacast_wip, val_cols, id_var, dimension_var, suffix_dim, suffix_specs):
  """
    A function to aggregate unacast data to unique id level - prerequisite to hex id aggregation (accounting for proportions and sub categories of temporal columns).
    
    Arguments: 
    carto_unacast_wip {gpd.DataFrame} -- Carto Unacast data post initial spread of return rate columns 
    val_cols {str list} -- Names of value columns to be aggregated
    id_var {string} -- Name of dataset key id var/quadid (indicating unique rows, defaults to "seen_in")
    dimension_var {string} -- Name of temporal column to split the categories of (i.e. "day_of_week", "month", "time_of_day")
    suffix_dim {string} -- Name of suffix corresponding to dimension_var (marks source data in new cols)
    suffix_specs {str list} -- Names of dimension_var categories (pssible values to be spread into new variables) 
 
    Returns: Aggregated GeoDataFrame with additional variables derived from categories of temporal variables.
  """  
  
  suffixes = ["_" + each + suffix_dim for each in suffix_specs]
  rr_cols = [col for col in val_cols if col.startswith("return_rate_")]
  val_cols_pure = list(set(val_cols) - set(rr_cols))
  
  # Aggregations:
  
  # 1) reflect temporal categories by pivoting with sum of value columns
  output = carto_unacast_wip.pivot_table(values=val_cols_pure, columns=dimension_var, index=id_var, aggfunc=dict(zip(val_cols_pure, ["sum"]*len(val_cols_pure))))
  fix_pivot_data_names(data=output, suffix=suffix_dim, id_var=id_var)
  
  # 1.1) keep sum of the [avg number of people] per subcategory
  out_ppl_sum = output.copy().set_index(id_var)
  out_ppl_sum = out_ppl_sum.loc[:, out_ppl_sum.columns.str.startswith("proportion_")].reset_index()
  out_ppl_sum.columns = out_ppl_sum.columns.str.replace(r"proportion_", "sum_")
  
  # 1.2) add columns reverted back to proportions 
  for suffix in suffixes:
  
    columns_with_suffix = [col for col in output.columns if suffix in col]
    denominator=[col for col in columns_with_suffix if col.startswith("staying_")]
    numerators=list(set(columns_with_suffix) - set(denominator))
 
    for numerator in numerators:
      output[numerator] = output[numerator]/output[denominator[0]]
      
  # 2) build combined output file
  output = output. \
    merge(out_ppl_sum, on=id_var, how="left")
    
  return output


#### 4. Functions on combining the TAA components

In [32]:
def merge_taa_components_from_wppop(pop_stats, poi_stats, carto_cexpend_stats, carto_sociodemo_stats, carto_unacast_stats, carto_poi_stats,
                                    id_var="hex_id", export_spec=None):
  """
    A function to combine trade-area-analysis component datasets.
    
    Arguments: 
    pop_stats {GeoDataFrame} -- WP population data - aggregated to hexagon level
    poi_stats {GeoDataFrame} -- OSM poi data - aggregated to hexagon level
    carto_cexpend_stats {GeoDataFrame} -- Carto customer spend data - aggregated to hexagon level
    carto_sociodemo_stats {GeoDataFrame} -- Carto socio-demographic data - aggregated to hexagon level
    carto_unacast_stats {GeoDataFrame} -- Carto Unacast data - aggregated to hexagon level
    carto_poi_stats {GeoDataFrame} -- Carto PB poi data - aggregated to hexagon level
    id_var {string} -- Name of id variable to merge on
    export_spec {string} -- Name of file to export to blob (path and name)
 
    Returns: GeoDataFrame with wider data.
  """   
  
  # need a dataset with geometry and hex_id-> to form a gpd
  # currently using pop_stats
  
  output = \
    pop_stats. \
    merge(poi_stats, on=id_var, how='left'). \
    merge(carto_cexpend_stats, on=id_var, how='left'). \
    merge(carto_sociodemo_stats, on=id_var, how='left'). \
    merge(carto_unacast_stats, on=id_var, how='left'). \
    merge(carto_poi_stats, on=id_var, how='left')
  
  if export_spec is not None:
    output.to_file(export_spec, driver="GeoJSON")
  
  return(output)


In [33]:
def merge_taa_components_from_osm_carto(poi_stats, carto_cexpend_stats, carto_sociodemo_stats, carto_unacast_stats, carto_poi_stats,
                                    id_var="hex_id", export_spec="/dbfs/mnt/AA/ba008/assets/trade_area_analysis/taa_combo_set.json"):
  """
    A function to combine trade-area-analysis component datasets.
    
    Arguments: 
    poi_stats {GeoDataFrame} -- OSM poi data - aggregated to hexagon level
    carto_cexpend_stats {GeoDataFrame} -- Carto customer spend data - aggregated to hexagon level
    carto_sociodemo_stats {GeoDataFrame} -- Carto socio-demographic data - aggregated to hexagon level
    carto_unacast_stats {GeoDataFrame} -- Carto Unacast data - aggregated to hexagon level
    carto_poi_stats {GeoDataFrame} -- Carto PB poi data - aggregated to hexagon level
    id_var {string} -- Name of id variable to merge on
    export_spec {string} -- Name of file to export to blob (path and name)
 
    Returns: GeoDataFrame with wider data.
  """   
  
  # need a dataset with geometry and hex_id-> to form a gpd
  # currently using pop_stats
  output = \
    poi_stats. \
    merge(carto_cexpend_stats, on=id_var, how='left'). \
    merge(carto_sociodemo_stats, on=id_var, how='left'). \
    merge(carto_unacast_stats, on=id_var, how='left'). \
    merge(carto_poi_stats, on=id_var, how='left')
  
  output = gpd.GeoDataFrame(output)
  output.to_file(export_spec, driver="GeoJSON")
  
  return(output)


In [34]:
def merge_taa_components_from_hex(territory_hex, poi_stats, carto_cexpend_stats, carto_sociodemo_stats, carto_unacast_stats, carto_poi_stats,
                                  id_var="hex_id", export_spec="/dbfs/mnt/AA/ba008/assets/trade_area_analysis/taa_combo_set.json"):
  """
    A function to combine trade-area-analysis component datasets.
    
    Arguments: 
    territory_hex {GeoDataFrame} -- Hexagonal grid for the territory
    poi_stats {GeoDataFrame} -- OSM poi data - aggregated to hexagon level
    carto_cexpend_stats {GeoDataFrame} -- Carto customer spend data - aggregated to hexagon level
    carto_sociodemo_stats {GeoDataFrame} -- Carto socio-demographic data - aggregated to hexagon level
    carto_unacast_stats {GeoDataFrame} -- Carto Unacast data - aggregated to hexagon level
    carto_poi_stats {GeoDataFrame} -- Carto PB poi data - aggregated to hexagon level
    id_var {string} -- Name of id variable to merge on
    export_spec {string} -- Name of file to export to blob (path and name)
 
    Returns: GeoDataFrame with wider data.
  """   
  
  # need a dataset with geometry and hex_id-> to form a gpd
  # currently using territory_hex
  
  output = \
    territory_hex. \
    merge(poi_stats, on=id_var, how='left'). \
    merge(carto_cexpend_stats, on=id_var, how='left'). \
    merge(carto_sociodemo_stats, on=id_var, how='left'). \
    merge(carto_unacast_stats, on=id_var, how='left'). \
    merge(carto_poi_stats, on=id_var, how='left')
 
  output.to_file(export_spec, driver="GeoJSON")
  
  return(output)

#### Data visualization functions

In [35]:
def plot_simple_hex(combo_set, show_var):
  """
    A function to display data per hexagon - on regional and city level.
    
    Arguments: 
    combo_set {gpd.DataFrame} -- Complete data, on regional level
    show_var {string} -- Name of variable to display
 
    Returns: GeoDataFrame with wider data.
  """    
  
  combo_set.plot(column=show_var, figsize=(10, 10))  
